This scripts contains of 7 steps: 
    - STEP 1: Import packages
    - Step 2: Setting the cnx
    - Step 3: Definitions for reading in the dataframes
    - Step 4: Constant values for visualisations
    - Step 5: Definitions for visualisations
    - Step 6: Definition for the constantly updating of dataframes / visualisations
    - Step 7: Implementation of all above mentioned steps

It's important to have step 7 running right before you start Simio. 

In [ ]:
#STEP 1: Import packages
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import geopandas
import geopandas as gpd
from geopandas.geodataframe import GeoDataFrame
from shapely.geometry import Point
from mpl_toolkits.basemap import Basemap
from heapq import nlargest

#import mySQL option 1
import mysql.connector

#import mySQL option 2
#import pymysql.cursors #only try this after running the "python -m pip install PyMySQL" 

In [ ]:
#STEP 2: Setting the cnx
#cnx for option 1
cnx = mysql.connector.connect(user='epa1351g14', password='xgt65RR##', database='epa1351group14')
#cnx for option 2
#cnx = pymysql.connect(user='epa1351g14', password='xgt65RR##', database='epa1351group14')

Definitions for dataframes

In [ ]:
#STEP 3a: Definition for reading in dataframe of segmenttraveltimes
def read_stt_sql(cnx):
    cursor = cnx.cursor()
   
    query_stt = ("SELECT * from segmenttraveltimes ORDER BY RowID2")
    segmenttraveltimes = pd.read_sql(query_stt,cnx)
    
    return segmenttraveltimes

In [ ]:
#STEP 3b: Definition for reading in dataframe of brokenbridges
def read_bb_sql(cnx):
    cursor = cnx.cursor()

    query_bb = ("SELECT * from brokenbridges ORDER BY RowID3")
    brokenbridges = pd.read_sql(query_bb,cnx)
    
    return brokenbridges

Definitions for visualizations

In step 4 constant values for visualisations are inserted. 
IMPORTANT NOTE: Re-run step 4a & 4b, because the lists needs to be reset!

In [ ]:
#Step 4a: Constant values for visualisations for table segmenttraveltimes
LN1 = list()
LDC = list()
N1DC = pd.read_csv('Data_clean_N1_Dhaka_Chittagong_new.csv')
LRPsDhakaChittagong = list(N1DC.LRPName.unique())

In [ ]:
#Step 4b: Constant values for visualisations for table brokenbridges
bridges = pd.read_csv('Bridge_data_with_link_to_traffic.csv')
shape = gpd.read_file('bgd_admbnda_adm0_bbs_20180410/bgd_admbnda_adm0_bbs_20180410.shp')
bridges=bridges[bridges['road']=='N1']
#Make coordinates from the latitude and longitude
bridges['Coordinates'] = list(zip(bridges.lon, bridges.lat))
bridges['Coordinates'] = bridges['Coordinates'].apply(Point)
bridges = GeoDataFrame(bridges, geometry='Coordinates')
shape.crs = {'init': 'epsg:4326'}
bridges.crs = {'init': 'epsg:4326'}
fig, ax = plt.subplots(1, figsize=(3.5,7))
base = shape.plot(ax=ax, color='#DCDCDC')
bridges.plot(ax=base, marker='o', alpha=0.5, color = 'green', markersize = 1)

In [ ]:
#Step 5a: Definition for the moving average of the travel time over the N1 & Dhaka - Chittagong
def movingaverage(segmenttraveltimes):
    dfma = segmenttraveltimes[segmenttraveltimes.LRPName.isin(LRPsDhakaChittagong)]
    LN1.append(segmenttraveltimes.SegTime.sum()/len(segmenttraveltimes))
    LDC.append(dfma.SegTime.sum()/len(dfma))
    x = list(range(1,len(LN1)+1))

    plt.plot(x, LN1, label = 'average travel time N1')
    plt.plot(x, LDC, label = 'average travel time Dhaka - Chittagong')
    plt.title('Moving average of travel time ')
    plt.xlabel('Simulation hours')
    plt.ylabel('Travel time (hours)')
    plt.legend( loc=(1.02, 0.7) )
    plt.show()
    return

In [ ]:
#Step 5b: Definition for the vehicletypes over the top 8 bussiest segments on the N1
def vehiclespersegment(segmenttraveltimes):
    n = 8
    segments = segmenttraveltimes.LRPName.unique()
    vehicles = segmenttraveltimes.VehicleType.unique()
    segmentdict = {}
    for segment in segments:
        df = segmenttraveltimes[segmenttraveltimes.LRPName == segment]
        vehicledict = {}
        for vehicle in vehicles:
            df2 = df[df.VehicleType == vehicle]
            segtimepervehicle = df2.SegTime.sum()
            vehicledict[vehicle] = segtimepervehicle
            segmentdict[segment] = vehicledict

    newdict = {}
    for key, value in segmentdict.items():
        newdict[key] = sum(segmentdict[key].values())
    highestsegment = nlargest(n, newdict, key=newdict.get)
    highestsegmentdict = {}
    for segment in highestsegment:
        highestsegmentdict[segment] = segmentdict[segment]

    df = pd.DataFrame(highestsegmentdict)
    
    df.plot(kind="bar", stacked=True)
    plt.legend(bbox_to_anchor=(1.1, 1.05))
    plt.title('Vehicletypes over top 8 bussiest segments')
    plt.xlabel('Vehicletype')
    plt.ylabel('Number of vehicles')
    plt.legend( loc=(1.02, 0.7) )
    plt.show()
    return

In [ ]:
#Step 5c: Definition for the top 8 bussiest segments shown per vehicletype
def vehicletypes(segmenttraveltimes):
    n = 8
    segmenttraveltimes[segmenttraveltimes.LRPName.isin(list(segmenttraveltimes.groupby('LRPName')['SegTime'].sum().nlargest(n).to_dict().keys()))].groupby(['LRPName','VehicleType'])['SegTime'].sum().unstack().plot(kind='bar', stacked=True).legend(bbox_to_anchor=(1.1, 1.05))
    
    plt.title('Top 8 bussiest segments shown for the vehicletypes')
    plt.xlabel('Segments')
    plt.ylabel('Number of vehicles')
    plt.legend( loc=(1.02, 0.7) )
    plt.show()
    return

In [ ]:
#Step 5d: Definition for the currently broken bridges on the N1
def bridgesdown(brokenbridges):
    bridgesbrokendown = brokenbridges.BridgeID.unique()
    brokenbridges = bridges[bridges.LRPName.isin(bridgesbrokendown)]
    brokenbridges.plot(ax=base, marker = 'o', alpha = 0.5, color = 'red', markersize = 100)
    ax.set_title("Bridges broken down in red")
    return

Definition for the updating visualisations

In [ ]:
#STEP 6: Definition to update all the visualisations
def wait_for_sql():
    last_rowid = -1
    while True:
        #cnx for option 1
        cnx = mysql.connector.connect(user='epa1351g14', password='xgt65RR##', database='epa1351group14')
        #cnx for option 2
        #cnx = pymysql.connect(user='epa1351g14', password='xgt65RR##', database='epa1351group14')
        cursor = cnx.cursor()
        
        #selecting the latest row in hourlyupdate table
        query = ("SELECT * from hourlyupdate ORDER BY RowID1 DESC LIMIT 1")
        cursor.execute(query)

#for loop searching for a new row in the hourlyupdate table        
        for (RowID1, SimioHour) in cursor:
            if RowID1 > last_rowid:
                clear_output()
                
                #Reading in the dataframes
                read_stt_sql(cnx)
                segmenttraveltimes = read_stt_sql(cnx)
                read_bb_sql(cnx)
                brokenbridges = read_bb_sql(cnx)
                
                #Visualisations
                movingaverage(segmenttraveltimes)
                vehiclespersegment(segmenttraveltimes)
                vehicletypes(segmenttraveltimes)
                #bridgesdown(brokenbridges)
                
                #change the value of last_rowid to the current Row ID
                last_rowid = RowID1
        time.sleep(1)

In [ ]:
wait_for_sql()